# Langchain SQL Agent

In [ ]:
# %pip install langchain-openai
# %pip install langchain
# %pip install --upgrade --quiet  langchain langchain-community langchain-experimental

In [68]:
import sqlite3
import pandas as pd
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain.sql_database import SQLDatabase

In [110]:
from langchain.llms import LlamaCpp
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain.callbacks.manager import CallbackManager
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

## Loading the SQL database

In [86]:
db = SQLDatabase.from_uri("sqlite:////kaggle/input/chinook-database/Chinook.db")

In [87]:
print(db.dialect)

sqlite


In [88]:
print(db.get_usable_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [97]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [106]:
db.run("SELECT * FROM Customer LIMIT 10;")

"[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4), (6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5), (7, 'As

In [107]:
db.run("SELECT * FROM InvoiceLine LIMIT 10;")

'[(1, 1, 2, 0.99, 1), (2, 1, 4, 0.99, 1), (3, 2, 6, 0.99, 1), (4, 2, 8, 0.99, 1), (5, 2, 10, 0.99, 1), (6, 2, 12, 0.99, 1), (7, 3, 16, 0.99, 1), (8, 3, 20, 0.99, 1), (9, 3, 24, 0.99, 1), (10, 3, 28, 0.99, 1)]'

## Setting up the Langchain SQL Agent

In [ ]:
with open('/kaggle/input/api-key-open-ai/open_ai_api_key.txt', 'r') as file:
    api_key = file.read()

In [98]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0125',
                 temperature=0, 
                 openai_api_key=api_key)

In [99]:
# Create a SQL database toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [100]:
toolkit.get_tools

<bound method SQLDatabaseToolkit.get_tools of SQLDatabaseToolkit(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fe9fd20c310>, llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe9fc4736a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe9fc514ca0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''))>

In [103]:
# Use SQLDatabaseToolkit within an Agent
agent_executor = create_sql_agent(llm=llm, 
                                  toolkit=toolkit, 
                                  agent_type='openai-tools',#AgentType.OPENAI_FUNCTIONS
                                  verbose=True)

In [102]:
agent_executor.invoke("""List the total sales per country. 
                      Which country's customers spent the most?""")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	Sã

{'input': "List the total sales per country. \n                      Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n\nThe country whose customers spent the most is the USA with a total sales amount of $523.06.'}

In [108]:
agent_executor.invoke("""How many individual customers in City:Prague bought Artist: AC/DC?""")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine, Track, Artist'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
Custo

{'input': 'How many individual customers in City:Prague bought Artist: AC/DC?',
 'output': 'There are 0 individual customers in Prague who bought music from the artist AC/DC.'}

In [109]:
agent_executor.invoke("""Sort the Countries by number of customers.""")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José dos Campos	SP	B

{'input': 'Sort the Countries by number of customers.',
 'output': 'The countries sorted by the number of customers are as follows:\n1. USA - 13 customers\n2. Canada - 8 customers\n3. France - 5 customers\n4. Brazil - 5 customers\n5. Germany - 4 customers\n6. United Kingdom - 3 customers\n7. Portugal - 2 customers\n8. India - 2 customers\n9. Czech Republic - 2 customers\n10. Sweden - 1 customer\n\n(Note: Only the top 10 countries are shown)'}

## SQL Chains with open-source LLM
https://stackoverflow.com/questions/76661319/using-an-open-source-llm-in-sql-chain

In [113]:
%pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 32.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.44-cp310-cp310-manylinux_2_31_x86_64.whl size=2538298 sha256=2d3522b1da2e01683bb9e68352c4c0eaa2162f5048f5ab53b41bd8d0de9b502c
  Stored in directory: /root/.cache/pip/wheels/6e/f0/52/1716aa7fefc7eb2a9b76775b0a61fc131b7dcc961e310a048a
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


In [114]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm_open_source = LlamaCpp(
    model_path = "codellama-13b-instruct.gguf",
    temperature = 0.0,
    n_ctx=10000,
    n_gpu_layers= 15, # Change this until this crash (This is for offloading to GPU)
    n_threads = 3,
    n_batch=512,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    repeat_penalty=1.8
)

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: codellama-13b-instruct.gguf. Received error Model path does not exist: codellama-13b-instruct.gguf (type=value_error)

In [ ]:
chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": "Sort the Countries by number of customers."})
print(response)
       
db.run(response)

# Db Chain method

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
db_chain.run("Sort the Countries by number of customers.")

I was not able to load LLama gguf file into Kaggle notebooks.